In [572]:
from logic.getting import *
from logic.scaler import *
from logic.supporting_functions import *
from logic.final_preproc_function import *
from logic.windows_preproc import *
from logic.stats import var
from logic.gbm import gbm
from logic.portfolio import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import cvxpy as cp
from pypfopt.risk_models import risk_matrix, fix_nonpositive_semidefinite, cov_to_corr, CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier


# Initialise

In [449]:
stocks = ['AAPL', 'AMT', 'MSFT', 'CAT', 'GS', 'JNJ', 'MCD', 'NEE', 'PG', 'XOM']
stocks_df = [0 for i in range(len(stocks))]
X_df= [0 for i in range(10)]
y_df = [0 for i in range(10)]

for i in range(len(stocks)):
    stock_raw = get_price_data_raw(stocks[i])
    stock_tech = get_technical_analysis(stocks[i])
    stock_macro = get_macro_data()
    X, y, log_df = create_x_y(stock_raw, stock_tech, stock_macro)
    rename_columns(stock_raw)
    stock = stock.sort_values(by='datetime')
    stocks_df[i] = stock_raw 
    X_df[i] = X
    y_df[i] = y

/root/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/root/code/oscarthse/portfolio_manager/logic/supporting_functions.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{prefix}_1'] = (df[column_name] == 1).astype(int)
/root/code/oscarthse/portfolio_manager/logic/supporting_functions.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

/root/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/root/code/oscarthse/portfolio_manager/logic/supporting_functions.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{prefix}_1'] = (df[column_name] == 1).astype(int)
/root/code/oscarthse/portfolio_manager/logic/supporting_functions.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [148]:
y_df[0]

,Category_1,Category_0,Category_-1
1,1,0,0
2,1,0,0
3,1,0,0
4,0,0,1
5,1,0,0
...,...,...,...
139305,0,1,0
139306,0,1,0
139307,0,1,0
139308,0,1,0


# Average returns

In [100]:
percentages = [0 for i in range(len(stocks_df))]

for i in range(len(stocks_df)):
    stocks_df[i]['percentage_increase'] = (stocks_df[i]['close'] / stocks_df[i]['close'].shift(1) - 1) * 100
    percentages[i] = np.array(stocks_df[i]['percentage_increase'].dropna())
    percentages[i].sort()

In [117]:
stocks

['AAPL', 'AMT', 'MSFT', 'CAT', 'GS', 'JNJ', 'MCD', 'NEE', 'PG', 'XOM']

In [118]:
percentages

aapl_high = np.array([per for per in percentages[0] if per > 0.07]).mean()
aapl_low = np.array([per for per in percentages[0] if per < -0.07]).mean()

amt_high = np.array([per for per in percentages[1] if per > 0.07]).mean()
amt_low = np.array([per for per in percentages[1] if per < -0.07]).mean()

msft_high = np.array([per for per in percentages[2] if per > 0.07]).mean()
msft_low = np.array([per for per in percentages[2] if per < -0.07]).mean()

cat_high = np.array([per for per in percentages[3] if per > 0.07]).mean()
cat_low = np.array([per for per in percentages[3] if per < -0.07]).mean()

gs_high = np.array([per for per in percentages[4] if per > 0.07]).mean()
gs_low = np.array([per for per in percentages[4] if per < -0.07]).mean()

jnj_high = np.array([per for per in percentages[5] if per > 0.07]).mean()
jnj_low = np.array([per for per in percentages[5] if per < -0.07]).mean()

mcd_high = np.array([per for per in percentages[6] if per > 0.07]).mean()
mcd_low = np.array([per for per in percentages[6] if per < -0.07]).mean()

nee_high = np.array([per for per in percentages[7] if per > 0.07]).mean()
nee_low = np.array([per for per in percentages[7] if per < -0.07]).mean()

pg_high = np.array([per for per in percentages[8] if per > 0.07]).mean()
pg_low = np.array([per for per in percentages[8] if per < -0.07]).mean()

xom_high = np.array([per for per in percentages[9] if per > 0.07]).mean()
xom_low = np.array([per for per in percentages[9] if per < -0.07]).mean()

In [295]:
returns = {}
returns['AAPL'] = {'up': aapl_high, 'down': aapl_low}
returns['AMT'] = {'up': amt_high, 'down': amt_low}
returns['MSFT'] = {'up': msft_high, 'down': msft_low}
returns['CAT'] = {'up': cat_high, 'down': cat_low}
returns['GS'] = {'up': gs_high, 'down': gs_low}
returns['JNJ'] = {'up': jnj_high, 'down': jnj_low}
returns['MCD'] = {'up': mcd_high, 'down': mcd_low}
returns['NEE'] = {'up': nee_high, 'down': nee_low}
returns['PG'] = {'up': pg_high, 'down': pg_low}
returns['XOM'] = {'up': xom_high, 'down': xom_low}

In [296]:
returns

{'AAPL': {'up': 0.31837599446778025, 'down': -0.320746238418792},
 'AMT': {'up': 0.38715191527494913, 'down': -0.3838609516499593},
 'MSFT': {'up': 0.3012643276062388, 'down': -0.3006000349963165},
 'CAT': {'up': 0.3806318467465596, 'down': -0.3847502673804319},
 'GS': {'up': 0.3720395815897206, 'down': -0.37221715372579733},
 'JNJ': {'up': 0.2727440579193454, 'down': -0.27027235775281816},
 'MCD': {'up': 0.2981475520316021, 'down': -0.29351101170426297},
 'NEE': {'up': 0.33643489556861783, 'down': -0.33844376135138654},
 'PG': {'up': 0.27878488059638934, 'down': -0.2763514347692313},
 'XOM': {'up': 0.3088787675541417, 'down': -0.31336584097404435}}

In [126]:
print(nee_low)

-0.33844376135138654


# Models

In [150]:
models

[<Sequential name=sequential_5, built=True>,
 <Sequential name=sequential_4, built=True>,
 <Sequential name=sequential_6, built=True>,
 <Sequential name=sequential_6, built=True>,
 <Sequential name=sequential_3, built=True>,
 <Sequential name=sequential_7, built=True>,
 <Sequential name=sequential_10, built=True>,
 <Sequential name=sequential_5, built=True>,
 <Sequential name=sequential_3, built=True>,
 <Sequential name=sequential_8, built=True>]

In [232]:
def calculated_return(model, X_pred, up, down):
    predictions = model.predict(X_pred)
    cat_1 = predictions[0][0]
    cat_2 = predictions[0][2]
    return cat_1*up + cat_2*down

In [513]:
X_fit = [X_df[i] for i in range(10)]
for i in range(10):
    stocks_df[i] = stocks_df[i].sort_values(by = 'datetime')
    stocks_df[i] = stocks_df[i].reset_index()
    stocks_df[i] = stocks_df[i].drop(columns='index')
    stocks_df[i] = stocks_df[i].merge(merged, how = 'inner', on = 'datetime')
    stocks_df[i]['combo'] = stocks_df[i]['volume'] - stocks_df[i]['close']
    X_fit[i]['combo'] = X_fit[i]['volume'] - X_fit[i]['close']
    X_fit[i] = stocks_df[i].filter(['datetime', 'combo']).merge(X_fit[i], how = 'inner', on = 'combo')
    X_fit[i] = X_fit[i].sort_values(by = 'datetime')
    X_fit[i] = X_fit[i].reset_index(drop = True)
    X_fit[i] = X_fit[i].drop(columns = ['combo', 'datetime'])

In [573]:
X_fit[9]

,open,high,low,close,volume,EMA,SlowK,SlowD,Real Upper Band,Real Middle Band,...,RSI,OBV,SAR,interest_rate,GDP,inflation,cos_time_of_day,sin_time_of_day,cos_day_of_year,sin_day_of_year
0,26.449,26.601,26.163,26.327,829800,26.4059,57.0579,59.3514,26.7354,26.3877,...,42.7234,-2641300.0,25.9334,2.25,3867.076,3.392747,-8.660254e-01,-0.500000,0.994671,0.103102
1,26.396,26.526,26.121,26.301,704800,26.4056,48.5571,54.9994,26.7183,26.3822,...,42.0411,-3346100.0,25.9678,2.25,3867.076,3.392747,-7.933533e-01,-0.608761,0.994671,0.103102
2,26.375,26.574,26.137,26.385,1185200,26.4046,48.3109,51.3086,26.7015,26.3765,...,41.7723,-2160900.0,26.0005,2.25,3867.076,3.392747,-7.071068e-01,-0.707107,0.994671,0.103102
3,26.454,26.622,26.210,26.390,1329500,26.4062,50.3101,49.0593,26.6933,26.3738,...,43.1625,-831400.0,26.0316,2.25,3867.076,3.392747,-6.087614e-01,-0.793353,0.994671,0.103102
4,26.460,27.046,26.163,26.395,356100,26.4080,46.5559,48.3923,26.6828,26.3706,...,43.2672,-475300.0,26.0611,2.25,3867.076,3.392747,-5.000000e-01,-0.866025,0.994671,0.103102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68389,109.129,109.165,109.102,109.143,361,108.7591,68.1512,62.4399,109.8237,108.8244,...,55.1487,-10573762.0,109.7540,5.33,5642.697,4.116338,-2.588190e-01,-0.965926,-0.504961,-0.863142
68390,109.128,109.244,109.083,109.094,17241,108.7712,65.6710,66.0706,109.8203,108.8438,...,55.1429,-10591003.0,109.7406,5.33,5642.697,4.116338,-1.305262e-01,-0.991445,-0.504961,-0.863142
68391,109.108,109.126,109.083,109.104,2361,108.7823,63.9827,65.9350,109.8117,108.8638,...,55.0263,-10588642.0,109.7274,5.33,5642.697,4.116338,-1.836970e-16,-1.000000,-0.504961,-0.863142
68392,109.177,109.234,109.083,109.212,5550871,108.7952,55.3463,61.6667,109.7971,108.8865,...,55.3575,-5037771.0,109.7143,5.33,5642.697,4.116338,1.305262e-01,-0.991445,-0.504961,-0.863142


In [545]:
X_pred = X_fit[0][40001:40002]

In [546]:
scale = initialize_scaler(X_fit[0])
X_pred = transform_scaler(scale, X_pred)

In [547]:
models[0].predict(X_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


array([[0.40258875, 0.33247897, 0.26493227]], dtype=float32)

In [548]:
calculated_return(models[0], X_pred, returns['AAPL']['up'], returns['AAPL']['down'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


0.04319856461665511

In [549]:
df_test[0]

-0.320746238418792

In [557]:
df_test
# ret = df_test.T['2005-01-03 09:30:00+00:00']

[0.04319856461665511,
 0.2024508107083475,
 0.16562913775455676,
 0.17484913028134746,
 0.21354789118284412,
 0.15051216180335103,
 0.18270988269034638,
 0.21735087313114565,
 0.09010098823983828,
 0.2829762647364938]

In [556]:
df_test = X_fit.copy()
for i in range(10):
    df_test[i] = calculated_return(models[i], X_pred, returns[stocks[i]]['up'], returns[stocks[i]]['down'])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


# Optimize

In [568]:
stocks_df[0]

,datetime,open,high,low,close,volume,combo
0,2005-01-03 09:30:00+00:00,0.980,0.986,0.953,0.962,178229296,178229295.038
1,2005-01-03 10:00:00+00:00,0.962,0.965,0.947,0.953,93294656,93294655.047
2,2005-01-03 10:30:00+00:00,0.953,0.956,0.945,0.954,83893656,83893655.046
3,2005-01-03 11:00:00+00:00,0.954,0.957,0.948,0.956,40442528,40442527.044
4,2005-01-03 11:30:00+00:00,0.956,0.962,0.953,0.958,36368976,36368975.042
...,...,...,...,...,...,...,...
68426,2023-08-31 17:00:00+00:00,187.646,189.439,187.345,187.644,74843,74655.356
68427,2023-08-31 17:30:00+00:00,187.606,187.691,187.385,187.660,74155,73967.34
68428,2023-08-31 18:00:00+00:00,187.646,187.721,187.465,187.630,18421,18233.37
68429,2023-08-31 18:30:00+00:00,187.666,187.771,187.565,187.580,14248,14060.42


In [566]:
df = stocks_df[0].filter(['datetime', 'close'])
for i in range(1, len(stocks)):
    df = df.merge(stocks_df[i].filter(['close', 'datetime']), on= 'datetime', how= 'inner')
    df = df.rename(columns = {'close_x': stocks[i-1]})
    df = df.rename(columns = {'close_y': stocks[i]})
    

In [559]:
df = df.set_index('datetime')

In [565]:
df

,AAPL,AMT,MSFT,CAT,GS,JNJ,MCD,NEE,PG,XOM
datetime,,,,,,,,,,
2005-01-03 09:30:00+00:00,0.962,14.397,18.761,29.603,78.755,36.730,18.986,5.228,32.511,26.769
2005-01-03 10:00:00+00:00,0.953,14.311,18.677,29.401,78.313,36.521,18.886,5.244,32.470,26.664
2005-01-03 10:30:00+00:00,0.954,14.358,18.683,29.350,78.380,36.515,18.786,5.223,32.499,26.643
2005-01-03 11:00:00+00:00,0.956,14.397,18.649,29.136,78.403,36.469,18.851,5.219,32.382,26.548
2005-01-03 11:30:00+00:00,0.958,14.374,18.656,29.084,78.515,36.446,18.939,5.216,32.341,26.580
...,...,...,...,...,...,...,...,...,...,...
2023-08-31 17:00:00+00:00,187.644,177.775,326.349,278.274,322.677,159.393,278.044,65.707,152.399,109.143
2023-08-31 17:30:00+00:00,187.660,178.089,326.369,278.402,322.125,159.167,277.906,65.648,152.399,109.094
2023-08-31 18:00:00+00:00,187.630,178.757,326.638,278.769,322.815,159.315,278.054,65.707,152.261,109.104


In [560]:
cov = risk_matrix(df)

In [561]:
cov

,AAPL,AMT,MSFT,CAT,GS,JNJ,MCD,NEE,PG,XOM
AAPL,0.007136,0.002168,0.002865,0.002954,0.003273,0.001194,0.001629,0.001488,0.001219,0.002073
AMT,0.002168,0.006564,0.002044,0.002433,0.002424,0.001374,0.001638,0.002147,0.001491,0.001798
MSFT,0.002865,0.002044,0.005330,0.002633,0.002732,0.001313,0.001551,0.001564,0.001351,0.001936
CAT,0.002954,0.002433,0.002633,0.007714,0.004104,0.001480,0.001910,0.001750,0.001455,0.003194
GS,0.003273,0.002424,0.002732,0.004104,0.008796,0.001511,0.001994,0.001850,0.001507,0.002950
JNJ,0.001194,0.001374,0.001313,0.001480,0.001511,0.002423,0.001090,0.001232,0.001240,0.001317
MCD,0.001629,0.001638,0.001551,0.001910,0.001994,0.001090,0.003383,0.001383,0.001185,0.001499
NEE,0.001488,0.002147,0.001564,0.001750,0.001850,0.001232,0.001383,0.004304,0.001410,0.001656
PG,0.001219,0.001491,0.001351,0.001455,0.001507,0.001240,0.001185,0.001410,0.002730,0.001282
XOM,0.002073,0.001798,0.001936,0.003194,0.002950,0.001317,0.001499,0.001656,0.001282,0.005229


In [574]:
df

,datetime,AAPL,AMT,MSFT,CAT,GS,JNJ,MCD,NEE,PG,XOM
0,2005-01-03 09:30:00+00:00,0.962,14.397,18.761,29.603,78.755,36.730,18.986,5.228,32.511,26.769
1,2005-01-03 10:00:00+00:00,0.953,14.311,18.677,29.401,78.313,36.521,18.886,5.244,32.470,26.664
2,2005-01-03 10:30:00+00:00,0.954,14.358,18.683,29.350,78.380,36.515,18.786,5.223,32.499,26.643
3,2005-01-03 11:00:00+00:00,0.956,14.397,18.649,29.136,78.403,36.469,18.851,5.219,32.382,26.548
4,2005-01-03 11:30:00+00:00,0.958,14.374,18.656,29.084,78.515,36.446,18.939,5.216,32.341,26.580
...,...,...,...,...,...,...,...,...,...,...,...
68426,2023-08-31 17:00:00+00:00,187.644,177.775,326.349,278.274,322.677,159.393,278.044,65.707,152.399,109.143
68427,2023-08-31 17:30:00+00:00,187.660,178.089,326.369,278.402,322.125,159.167,277.906,65.648,152.399,109.094
68428,2023-08-31 18:00:00+00:00,187.630,178.757,326.638,278.769,322.815,159.315,278.054,65.707,152.261,109.104
68429,2023-08-31 18:30:00+00:00,187.580,178.089,326.757,278.769,322.815,159.197,277.906,65.707,152.636,109.212


In [576]:
shr = CovarianceShrinkage(df, frequency = 252*20)
shr = shr.ledoit_wolf()

TypeError: cannot perform __truediv__ with this index type: DatetimeArray

In [575]:
shr

,AAPL,AMT,MSFT,CAT,GS,JNJ,MCD,NEE,PG,XOM
AAPL,0.142599,0.043213,0.057121,0.058894,0.065239,0.023799,0.032478,0.029655,0.024292,0.041323
AMT,0.043213,0.131196,0.040740,0.048510,0.048313,0.027387,0.032656,0.042799,0.029730,0.035841
MSFT,0.057121,0.040740,0.106592,0.052490,0.054468,0.026168,0.030910,0.031173,0.026937,0.038585
CAT,0.058894,0.048510,0.052490,0.154124,0.081808,0.029498,0.038083,0.034885,0.029004,0.063676
GS,0.065239,0.048313,0.054468,0.081808,0.175692,0.030114,0.039757,0.036886,0.030037,0.058801
JNJ,0.023799,0.027387,0.026168,0.029498,0.030114,0.048641,0.021724,0.024564,0.024721,0.026246
MCD,0.032478,0.032656,0.030910,0.038083,0.039757,0.021724,0.067790,0.027574,0.023625,0.029877
NEE,0.029655,0.042799,0.031173,0.034885,0.036886,0.024564,0.027574,0.086140,0.028103,0.033018
PG,0.024292,0.029730,0.026937,0.029004,0.030037,0.024721,0.023625,0.028103,0.054762,0.025554
XOM,0.041323,0.035841,0.038585,0.063676,0.058801,0.026246,0.029877,0.033018,0.025554,0.104587


In [571]:
ef = EfficientFrontier(df_test, shr)
ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose = True)

Expected annual return: 22.3%
Annual volatility: 21.1%
Sharpe Ratio: 0.96


(0.2231307241512243, 0.21098868463831233, 0.9627564838343889)

In [564]:
cleaned_weights

OrderedDict([('AAPL', 0.0),
             ('AMT', 0.04747),
             ('MSFT', 0.0),
             ('CAT', 0.0),
             ('GS', 0.0),
             ('JNJ', 0.15986),
             ('MCD', 0.19732),
             ('NEE', 0.22953),
             ('PG', 0.0),
             ('XOM', 0.36583)])

In [570]:
ef